In [1]:
pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 8.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 5.8 MB/s eta 0:00:00


In [3]:
pip install flash-attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 41.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.9 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.5.9.post1-cp310-cp310-linux_x86_64.whl size=120889689 sha256=5022ba11d48bf74926da9c16260f4ea2b9bb7f4e29bdb4bd6e1383ad1c55d16f
  Stored in directory: /root/.cache/pip/wheels/cc/ad/f6/7ccf0238790d6346e9fe622923a76ec218e890d356b9a2754a
Successfully built flash-attn


In [6]:
pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.6 MB/s eta 0:00:00


In [10]:
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline
import torch
import re

# IMPORT MODEL

In [11]:
# Load model in FP16 precision to save memory
chat_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype=torch.float16,  # change to auto
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

pipe = pipeline(
        "text-generation",
        model=chat_model,
        tokenizer=tokenizer,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# IMPORT DATA

In [99]:
# Sample research papers with authors
sample_papers = [
    {
        "title": "Advancements in Quantum Computing",
        "author": "Alice Johnson",
        "content": (
            "Quantum computing is revolutionizing the field of information technology by utilizing quantum bits (qubits) instead of traditional bits. "
            "This allows for unprecedented processing power and efficiency.\n\n"
            "Recent breakthroughs have demonstrated the feasibility of practical quantum computers. Companies like IBM and Google have developed quantum processors "
            "capable of performing complex calculations at speeds unattainable by classical computers.\n\n"
            "One major challenge is maintaining qubit coherence. Efforts are being made to improve error correction techniques, which are essential for the reliable "
            "operation of quantum computers. Advances in materials science have also contributed to more stable qubits.\n\n"
            "Quantum algorithms, such as Shor's algorithm for factoring large numbers, have the potential to disrupt industries that rely on encryption. These algorithms "
            "can solve certain problems exponentially faster than the best classical algorithms.\n\n"
            "The future of quantum computing holds immense promise, with potential applications ranging from drug discovery to financial modeling. As the technology matures, "
            "it is expected to have a profound impact on various fields.\n\n"
            "Further research is needed to overcome existing challenges and unlock the full potential of quantum computing. Collaborative efforts between academia, "
            "industry, and government agencies are essential to drive innovation and accelerate the development of practical quantum technologies.\n\n"
            "Moreover, quantum computing has implications for cybersecurity. Quantum-resistant cryptographic techniques are being developed to secure sensitive information "
            "against potential threats posed by quantum computers.\n\n"
            "In addition, quantum machine learning is emerging as a promising field. By harnessing the power of quantum algorithms, machine learning models can "
            "achieve unprecedented performance improvements, opening up new possibilities for data analysis and pattern recognition."
        )
    },
    {
        "title": "Machine Learning in Healthcare",
        "author": "Bob Smith",
        "content": (
            "Machine learning is transforming healthcare by enabling more accurate diagnoses and personalized treatments. It leverages large datasets to uncover patterns "
            "that are not apparent to human analysts.\n\n"
            "One key application is in medical imaging, where machine learning algorithms can detect anomalies in X-rays and MRIs with high precision. These tools assist radiologists "
            "in identifying early signs of diseases like cancer.\n\n"
            "Predictive analytics is another area where machine learning shines. By analyzing patient data, algorithms can predict the likelihood of disease outbreaks or patient readmissions, "
            "helping healthcare providers to take proactive measures.\n\n"
            "Natural language processing (NLP) allows for the analysis of unstructured data, such as doctors' notes and clinical reports. This capability enhances the understanding of patient "
            "histories and improves decision-making processes.\n\n"
            "The integration of machine learning in healthcare is not without challenges. Issues such as data privacy, algorithm bias, and the need for large, high-quality datasets must be addressed "
            "to fully realize the potential of these technologies.\n\n"
            "Moreover, machine learning is facilitating medical research by analyzing genomic data and identifying potential drug targets. This has the potential to revolutionize "
            "drug discovery and development processes, leading to more effective treatments for various diseases.\n\n"
            "Furthermore, machine learning algorithms are being used to optimize hospital operations, from resource allocation to patient scheduling. By streamlining administrative "
            "tasks, healthcare institutions can improve efficiency and reduce costs."
        )
    },
    {
        "title": "Renewable Energy Technologies",
        "author": "Carol White",
        "content": (
            "Renewable energy technologies are critical in the fight against climate change. They provide sustainable and cleaner alternatives to fossil fuels.\n\n"
            "Solar energy has seen significant advancements, with improvements in photovoltaic cell efficiency and reductions in cost. Large-scale solar farms are now viable sources of energy in many "
            "parts of the world.\n\n"
            "Wind energy is another growing sector. Innovations in turbine design have increased energy capture and reduced maintenance costs, making wind farms more economically competitive.\n\n"
            "Hydroelectric power remains a major renewable energy source. Advances in turbine technology and the development of small-scale hydro systems have expanded its applicability.\n\n"
            "Bioenergy and geothermal energy also contribute to the renewable energy mix. Research into biofuels and advanced geothermal systems continues to improve their efficiency and environmental impact.\n\n"
            "In addition to technological advancements, policy support and investment in renewable energy infrastructure are crucial for accelerating the transition to a sustainable energy future.\n\n"
            "Furthermore, renewable energy technologies offer economic benefits, such as job creation and energy independence. By investing in renewables, countries can reduce their reliance "
            "on imported fossil fuels and stimulate local economies."
        )
    },
    {
        "title": "Artificial Intelligence in Finance",
        "author": "David Brown",
        "content": (
            "Artificial intelligence (AI) is reshaping the finance industry by automating processes and enhancing decision-making. AI algorithms analyze vast amounts of data to identify trends and make predictions.\n\n"
            "In trading, AI systems execute trades at speeds and volumes impossible for humans, optimizing investment strategies and improving market efficiency.\n\n"
            "Risk management has also benefited from AI. Machine learning models assess credit risk and detect fraudulent transactions, providing greater security and reducing financial losses.\n\n"
            "Customer service is another area where AI has made an impact. Chatbots and virtual assistants handle customer inquiries, offering personalized support and freeing up human agents for more complex tasks.\n\n"
            "Despite the advantages, the integration of AI in finance raises concerns about job displacement, data privacy, and the potential for algorithmic bias. Addressing these challenges is crucial for the ethical deployment of AI technologies.\n\n"
            "Furthermore, AI-powered algorithms are being used to detect market trends and inform investment decisions. By analyzing vast amounts of financial data, these algorithms can identify "
            "profitable opportunities and mitigate risks.\n\n"
            "Moreover, AI-driven robo-advisors are gaining popularity among investors. These automated investment platforms use algorithms to create personalized portfolios based on individual "
            "financial goals and risk preferences."
        )
    },
    {
        "title": "Nanotechnology in Medicine",
        "author": "Eve Davis",
        "content": (
            "Nanotechnology is revolutionizing medicine by enabling the development of novel diagnostic and therapeutic tools. Nanoparticles can be engineered to target specific cells, enhancing the precision of treatments.\n\n"
            "One application of nanotechnology is in drug delivery. Nanocarriers can transport drugs directly to diseased cells, reducing side effects and improving treatment efficacy.\n\n"
            "Nanotechnology is also used in imaging. Nanoparticles can enhance the contrast in MRI and CT scans, allowing for earlier and more accurate detection of diseases.\n\n"
            "Tissue engineering benefits from nanotechnology through the creation of scaffolds that promote cell growth and tissue regeneration. These scaffolds mimic the natural extracellular matrix, providing a suitable environment for cell proliferation.\n\n"
            "While the potential of nanotechnology in medicine is vast, challenges such as toxicity, long-term effects, and regulatory hurdles need to be addressed to ensure safe and effective applications.\n\n"
            "Moreover, interdisciplinary collaboration between researchers, clinicians, and regulatory agencies is essential to accelerate the translation of nanotechnology-based medical innovations from the laboratory to clinical practice.\n\n"
            "In addition, nanotechnology holds promise for personalized medicine. By tailoring treatments to individual patients' genetic makeup and disease profiles, nanomedicine offers the potential for more effective and targeted therapies.\n\n"
            "Furthermore, nanotechnology-based diagnostic tools enable early disease detection and monitoring, improving patient outcomes and reducing healthcare costs.\n\n"
            "As nanotechnology continues to advance, its integration into clinical practice has the potential to revolutionize healthcare delivery and improve patient care."
        )
    },
    {
        "title": "The Future of Space Exploration",
        "author": "Emily Johnson",
        "content": (
            "Space exploration has long captured the imagination of humanity, driving technological innovation and expanding our understanding of the universe.\n\n"
            "Advancements in space propulsion systems are enabling faster and more efficient travel beyond Earth's orbit. Concepts such as ion propulsion and solar sails offer the potential "
            "for interstellar exploration and colonization.\n\n"
            "Robotic missions to celestial bodies, such as Mars and Europa, continue to uncover new insights into planetary geology and the potential for life beyond Earth.\n\n"
            "The emergence of commercial spaceflight companies, such as SpaceX and Blue Origin, is democratizing access to space and fostering innovation in space technology.\n\n"
            "Furthermore, international collaborations, such as the International Space Station (ISS), demonstrate the power of cooperation in advancing scientific research and exploration.\n\n"
            "As we look to the future, space exploration holds the promise of answering fundamental questions about our place in the universe and inspiring future generations of scientists and explorers."
        )
    },
    {
        "title": "Ethical Considerations in Genetic Engineering",
        "author": "Michael Wilson",
        "content": (
            "Genetic engineering offers unprecedented opportunities to modify and manipulate the genetic makeup of organisms, raising complex ethical questions and concerns.\n\n"
            "One of the key ethical considerations is the potential for unintended consequences. Genetic modifications intended to address one issue may have unforeseen impacts on ecosystems "
            "and biodiversity.\n\n"
            "Another concern is the possibility of genetic discrimination. Access to genetic enhancement technologies could exacerbate existing social inequalities and create new forms of discrimination based on genetic traits.\n\n"
            "The concept of 'designer babies,' where genetic traits are selected or modified for non-medical purposes, raises ethical dilemmas related to autonomy, consent, and the commodification of life.\n\n"
            "Moreover, the global nature of genetic engineering raises questions about governance and regulation. International collaboration and consensus-building are essential to develop "
            "ethical frameworks and guidelines for responsible genetic research and application.\n\n"
            "As genetic engineering technologies continue to advance, it is imperative that ethical considerations remain at the forefront of scientific discourse and decision-making processes."
        )
    },
    {
        "title": "The Rise of Virtual Reality",
        "author": "Sophia Miller",
        "content": (
            "Virtual reality (VR) technology is revolutionizing the way we interact with digital content and experience immersive environments.\n\n"
            "Advancements in VR hardware, such as headsets and haptic feedback devices, are making virtual experiences more realistic and engaging.\n\n"
            "VR has applications across various industries, including gaming, entertainment, education, and healthcare. In gaming, VR allows players to step into virtual worlds and "
            "interact with environments and characters in unprecedented ways.\n\n"
            "In healthcare, VR is being used for medical training, therapy, and pain management. Virtual simulations provide a safe and controlled environment for medical professionals "
            "to practice procedures and for patients to confront fears and phobias.\n\n"
            "The potential of VR extends beyond entertainment and healthcare. In education, VR enables immersive learning experiences, allowing students to explore historical events, "
            "scientific concepts, and cultural landmarks firsthand.\n\n"
            "As VR technology becomes more accessible and affordable, its impact on society is expected to grow, ushering in new forms of storytelling, communication, and collaboration."
        )
    },
    {
        "title": "The Impact of Climate Change on Biodiversity",
        "author": "Oliver Davis",
        "content": (
            "Climate change poses a significant threat to biodiversity, with profound implications for ecosystems and species around the world.\n\n"
            "Rising temperatures, changing precipitation patterns, and extreme weather events are altering habitats and disrupting ecosystems, leading to shifts in species distributions "
            "and changes in phenology.\n\n"
            "Many species are facing increased risk of extinction due to habitat loss, reduced food availability, and competition from invasive species.\n\n"
            "Marine ecosystems are particularly vulnerable to climate change, with ocean acidification, coral bleaching, and loss of sea ice threatening marine biodiversity.\n\n"
            "Addressing climate change requires urgent and coordinated action at the local, national, and global levels. Mitigation efforts, such as reducing greenhouse gas emissions and "
            "protecting and restoring habitats, are essential to safeguarding biodiversity.\n\n"
            "Adaptation strategies, such as creating climate-resilient protected areas and implementing sustainable land and water management practices, can help species and ecosystems "
            "adapt to changing environmental conditions.\n\n"
            "Preserving biodiversity is not only essential for maintaining healthy ecosystems and supporting human well-being but also crucial for mitigating the impacts of climate change "
            "and building resilience to future environmental challenges."
        )
    }
]

# PREPROCESS DATA

In [100]:
# Preprocess the content
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
    text = re.sub(r'\[[^]]*\]', '', text)  # Remove references
    return text.strip()

# Clean and chunk the text
documents = []
for paper in sample_papers:
    cleaned_content = clean_text(paper['content'])
    chunks = cleaned_content.split('. ')
    for chunk in chunks:
        if chunk.strip():  # Only keep non-empty chunks
            documents.append({'title': paper['title'], 'author': paper['author'], 'content': chunk.strip()})

# EMBED AND QUERY (Searching Database)

In [101]:
# Load the sentence transformer model
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for each document chunk
embeddings = []
metadata = []
for doc in documents:
    embedding = embed_model.encode(doc['content'])
    embeddings.append(embedding)
    metadata.append({'title': doc['title'], 'author': doc['author'], 'content': doc['content']})

# Convert embeddings list to numpy array
embeddings = np.array(embeddings)

# Create a FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# RETRIEVE CONTEXT

In [102]:
# Function to retrieve context based on query
def retrieve_context(query, k=5, threshold=1.3):
    query_embedding = embed_model.encode(query).reshape(1, -1)
    distances, indices = index.search(query_embedding, k)
    results = []
    for dist, idx in zip(distances[0], indices[0]):
        if dist < threshold:  # Only include relevant contexts
            results.append(metadata[idx])
    return results

In [103]:
# User query
query = "What is Nano Techology? And what can it do, sate your source"

# Retrieve context from the knowledge base
context = retrieve_context(query)
context_text = "\n\n".join([f"Title: {item['title']}\nAuthor: {item['author']}\nContent: {item['content']}" for item in context])

print(context_text)

Title: Nanotechnology in Medicine
Author: Eve Davis
Content: Nanotechnology is also used in imaging

Title: Nanotechnology in Medicine
Author: Eve Davis
Content: As nanotechnology continues to advance, its integration into clinical practice has the potential to revolutionize healthcare delivery and improve patient care.

Title: Nanotechnology in Medicine
Author: Eve Davis
Content: Moreover, interdisciplinary collaboration between researchers, clinicians, and regulatory agencies is essential to accelerate the translation of nanotechnology-based medical innovations from the laboratory to clinical practice

Title: Nanotechnology in Medicine
Author: Eve Davis
Content: In addition, nanotechnology holds promise for personalized medicine

Title: Nanotechnology in Medicine
Author: Eve Davis
Content: Nanotechnology is revolutionizing medicine by enabling the development of novel diagnostic and therapeutic tools


# GENERATE RESPONSE

In [110]:
generation_args = {
  "max_new_tokens": 500,
  "return_full_text": False,
  "temperature": 0.5,
  "do_sample": True,
}

def generate_response(instruction, query):
  context = retrieve_context(query)
  context_text = "\n\n".join([f"Title: {item['title']}\nAuthor: {item['author']}\nContent: {item['content']}" for item in context])

  # print(f"Context: {context_text}")

  messages = [{"role": "user", "content": f"{instruction} The Content: {context_text}"},
              {"role": "assistant", "content": "Welcome user, I am your helpful assistant"}
             ]

  input = {"role": "user", "content": query}
  messages.append(input)

  output = pipe(messages, **generation_args)
  response = output[0]['generated_text']

  print(f"Bot: {response}")
  # return response

  # print("\nHistory:")
  # for i in messages:
  #   print(f"Role: {i['role']} : {i['content']}")

In [114]:
q = "What are Nano Technology? And where can it be used (provide a list). Provide your author or source on where you got the data"
i = "You are a helpful assistant knowledgeable in various research topics. You are to output only the answer and nothing else. As much as possbile use the Context provided if there is context provided. Only use double space when necessary. If asked for source, do not reply with a link, provide the authors or anything related to the source related to the Content"

generate_response(i, q)

Bot:  Nanotechnology refers to the manipulation and control of matter on an atomic, molecular, and supramolecular scale, typically between 1 and 100 nanometers. It involves the design, characterization, production, and application of structures, devices, and systems by controlling shape and size at the nanometer scale.

Nanotechnology can be used in various fields, including but not limited to:

1. Medicine: Nanotechnology is used in drug delivery systems, medical imaging, diagnostics, tissue engineering, and regenerative medicine.
2. Electronics: Nanotechnology is used in the development of smaller and more efficient electronic devices, such as transistors, sensors, and memory storage.
3. Energy: Nanotechnology is used in the production of more efficient solar cells, batteries, and fuel cells.
4. Environment: Nanotechnology is used in water purification, air filtration, and pollution control.
5. Materials Science: Nanotechnology is used in the development of stronger, lighter, and mor

In [ ]:
i = "You are a helpful assistant knowledgeable in various research topics. You are to output only the answer and nothing else. As much as possbile use the Context provided if there is context provided. Only use double space when necessary"

while True:
  q = input("User: ")

  generate_response(i, q)